In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
#from keras.utils import to_categorical
#from sklearn.cluster import KMeans
import shutil
#import itertools

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
dataset = pd.read_csv('breastCancer.csv')# http://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Original)
dataset.iloc[:,0:11]#all rows, columns 0 up to 11

,id,clump_thickness,size_uniformity,shape_uniformity,marginal_adhesion,epithelial_size,bare_nucleoli,bland_chromatin,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
5,1017122,8,10,10,8,7,10,9,7,1,4
6,1018099,1,1,1,1,2,10,3,1,1,2
7,1018561,2,1,2,1,2,1,3,1,1,2
8,1033078,2,1,1,1,2,1,1,1,5,2
9,1033078,4,2,1,1,2,1,2,1,1,2


In [ ]:
dataset.describe()

In [ ]:
import seaborn as sns
sns.set(style="ticks")
sns.set_palette("Set1", n_colors=2)
sns.pairplot(dataset.iloc[:,1:11], hue="class")

In [3]:
# In CSV, label is the first column, after the features
CSV_COLUMNS = ['clump_thickness', 'size_uniformity','shape_uniformity','marginal_adhesion','epithelial_size','bare_nucleoli','bland_chromatin','normal_nucleoli','mitoses','class']
FEATURES = CSV_COLUMNS[0:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[9]
print(len(dataset))
dataset2 = dataset[dataset.bare_nucleoli != '?']#remove rows with bare_nucleoli='?'
print(len(dataset2))
dataset2 = dataset2.astype(int)
dataset2['class'][dataset2['class'] == 2] = 0 #convert class 2 to 0, benign
dataset2['class'][dataset2['class'] == 4] = 1 #convert class 4 to 1, malignant
#print(dataset2.dtypes)
df_train = dataset2.iloc[0:535,1:11] # use first 78% as training data
df_eval = dataset2.iloc[535:645,1:11] # use next 16% as eval data
df_test = dataset2.iloc[645:,1:10] # use last 6% as test data
print(FEATURES)
print(LABEL)
print(dataset2)
print(df_train.head())
print(df_train.tail())
print(df_train.dtypes)
df_test

699
683
['clump_thickness', 'size_uniformity', 'shape_uniformity', 'marginal_adhesion', 'epithelial_size', 'bare_nucleoli', 'bland_chromatin', 'normal_nucleoli', 'mitoses']
class
          id  clump_thickness  size_uniformity  shape_uniformity  \
0    1000025                5                1                 1   
1    1002945                5                4                 4   
2    1015425                3                1                 1   
3    1016277                6                8                 8   
4    1017023                4                1                 1   
5    1017122                8               10                10   
6    1018099                1                1                 1   
7    1018561                2                1                 2   
8    1033078                2                1                 1   
9    1033078                4                2                 1   
10   1035283                1                1                 1   
11   

,clump_thickness,size_uniformity,shape_uniformity,marginal_adhesion,epithelial_size,bare_nucleoli,bland_chromatin,normal_nucleoli,mitoses
661,4,1,1,1,2,1,3,1,1
662,1,1,3,1,2,1,2,1,1
663,1,1,3,1,2,1,2,1,1
664,3,1,1,3,2,1,2,1,1
665,1,1,1,1,2,1,1,1,1
666,5,2,2,2,2,1,1,1,2
667,3,1,1,1,2,1,3,1,1
668,5,7,4,1,6,1,7,10,3
669,5,10,10,8,5,5,7,10,1
670,3,10,7,8,5,8,7,4,1


In [4]:
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
  return input_columns

In [ ]:
feature_columns = make_feature_cols()
feature_columns

In [5]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = './logs/breastCancer_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

myopt = tf.train.FtrlOptimizer(learning_rate = 0.01)
model = tf.estimator.DNNClassifier(feature_columns = make_feature_cols(), 
                                   model_dir = OUTDIR, hidden_units=[10, 10], 
                                   n_classes=2,optimizer = myopt,
                                   activation_fn = tf.nn.relu)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './logs/breastCancer_trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1819164cf8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


![title](bcnn3.png)

In [6]:
def make_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    num_epochs = num_epochs,
    shuffle = True,
    num_threads = 1
  )

In [7]:
model.train(input_fn = make_input_fn(df_train, num_epochs = 3000))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./logs/breastCancer_trained/model.ckpt.
INFO:tensorflow:loss = 104.367195, step = 1
INFO:tensorflow:global_step/sec: 236.317
INFO:tensorflow:loss = 38.038002, step = 101 (0.426 sec)
INFO:tensorflow:global_step/sec: 350.604
INFO:tensorflow:loss = 38.628, step = 201 (0.285 sec)
INFO:tensorflow:global_step/sec: 344.942
INFO:tensorflow:loss = 26.135124, step = 301 (0.292 sec)
INFO:tensorflow:global_step/sec: 331.449
INFO:tensorflow:loss = 25.063044, step = 401 (0.300 sec)
INFO:tensorflow:global_step/sec: 361.394
INFO:tensorflow:loss = 17.861332, step = 501 (0.278 sec)
INFO:tensorflow:global_step/sec: 365.681
INFO:tensorflow:loss = 22.525223, step = 601 (0.272 sec)
INFO:tensorflow:global_step/sec: 362.641
INFO:te

INFO:tensorflow:global_step/sec: 333.243
INFO:tensorflow:loss = 10.154947, step = 8101 (0.300 sec)
INFO:tensorflow:global_step/sec: 355.541
INFO:tensorflow:loss = 8.627338, step = 8201 (0.281 sec)
INFO:tensorflow:global_step/sec: 287.504
INFO:tensorflow:loss = 7.1626167, step = 8301 (0.346 sec)
INFO:tensorflow:global_step/sec: 323.465
INFO:tensorflow:loss = 10.017387, step = 8401 (0.312 sec)
INFO:tensorflow:global_step/sec: 301.182
INFO:tensorflow:loss = 6.706417, step = 8501 (0.330 sec)
INFO:tensorflow:global_step/sec: 319.632
INFO:tensorflow:loss = 13.742269, step = 8601 (0.314 sec)
INFO:tensorflow:global_step/sec: 293.958
INFO:tensorflow:loss = 4.465642, step = 8701 (0.341 sec)
INFO:tensorflow:global_step/sec: 352.525
INFO:tensorflow:loss = 12.751464, step = 8801 (0.283 sec)
INFO:tensorflow:global_step/sec: 369.614
INFO:tensorflow:loss = 7.396802, step = 8901 (0.270 sec)
INFO:tensorflow:global_step/sec: 275.993
INFO:tensorflow:loss = 5.4717007, step = 9001 (0.362 sec)
INFO:tensorflo

In [8]:
ev = model.evaluate(input_fn = make_input_fn(df_eval, num_epochs = 1))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-05-16:00:57
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./logs/breastCancer_trained/model.ckpt-12540
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-05-16:00:58
INFO:tensorflow:Saving dict for global step 12540: accuracy = 0.9909091, accuracy_baseline = 0.76363635, auc = 0.99771065, auc_precision_recall = 0.9936931, average_loss = 0.049933, global_step = 12540, label/mean = 0.23636363, loss = 5.49263, prediction/mean = 0.23438235


In [10]:
print(ev)

{'accuracy': 0.9909091, 'accuracy_baseline': 0.76363635, 'auc': 0.99771065, 'auc_precision_recall': 0.9936931, 'average_loss': 0.049933, 'label/mean': 0.23636363, 'loss': 5.49263, 'prediction/mean': 0.23438235, 'global_step': 12540}


In [9]:
print('RMSE on {} dataset = {}'.format('validation', np.sqrt(ev['average_loss'])))
print('Accuracy on {} dataset = {}'.format('validation', np.sqrt(ev['accuracy'])))

RMSE on validation dataset = 0.22345693409442902
Accuracy on validation dataset = 0.9954441785812378


In [11]:
def make_test_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    num_epochs = num_epochs,
    shuffle = False,
    num_threads = 1
  )

In [15]:
prediction_result = model.predict(input_fn = make_test_fn(df_test, 1))

In [16]:
print(list(prediction_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./logs/breastCancer_trained/model.ckpt-12540
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[{'logits': array([-5.2220926], dtype=float32), 'logistic': array([0.00536706], dtype=float32), 'probabilities': array([0.99463296, 0.00536706], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object)}, {'logits': array([-5.699337], dtype=float32), 'logistic': array([0.00333701], dtype=float32), 'probabilities': array([0.9966629 , 0.00333701], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object)}, {'logits': array([-5.699337], dtype=float32), 'logistic': array([0.00333701], dtype=float32), 'probabilities': array([0.9966629 , 0.00333701], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object)}, {'logits': array([-4.759758], dtype=float32

In [21]:
predictions = list(model.predict(input_fn = make_test_fn(df_test, 1)))
predicted_classes = [p["classes"] for p in predictions]
probabilities = [p["probabilities"] for p in predictions]
#print(len(probabilities))
for x in range(len(probabilities)):
    print(
        "Class Prediction:    {}"
        .format(predicted_classes[x]))

    print("Probability:    {}\n"
        .format(probabilities[x]))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./logs/breastCancer_trained/model.ckpt-12540
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Class Prediction:    [b'0']
Probability:    [0.99463296 0.00536706]

Class Prediction:    [b'0']
Probability:    [0.9966629  0.00333701]

Class Prediction:    [b'0']
Probability:    [0.9966629  0.00333701]

Class Prediction:    [b'0']
Probability:    [0.9915051 0.0084949]

Class Prediction:    [b'0']
Probability:    [0.99604875 0.00395122]

Class Prediction:    [b'0']
Probability:    [0.9861596  0.01384038]

Class Prediction:    [b'0']
Probability:    [0.9959038  0.00409621]

Class Prediction:    [b'1']
Probability:    [0.06774748 0.9322525 ]

Class Prediction:    [b'1']
Probability:    [0.00584546 0.9941545 ]

Class Prediction:    [b'1']
Probability:    [0.10883514 0.8911649 ]

Class Prediction:    [b'0']
Prob

In [18]:
df_test

,clump_thickness,size_uniformity,shape_uniformity,marginal_adhesion,epithelial_size,bare_nucleoli,bland_chromatin,normal_nucleoli,mitoses
661,4,1,1,1,2,1,3,1,1
662,1,1,3,1,2,1,2,1,1
663,1,1,3,1,2,1,2,1,1
664,3,1,1,3,2,1,2,1,1
665,1,1,1,1,2,1,1,1,1
666,5,2,2,2,2,1,1,1,2
667,3,1,1,1,2,1,3,1,1
668,5,7,4,1,6,1,7,10,3
669,5,10,10,8,5,5,7,10,1
670,3,10,7,8,5,8,7,4,1


In [22]:
#save model
clump_thickness = tf.feature_column.numeric_column("clump_thickness");
size_uniformity = tf.feature_column.numeric_column("size_uniformity");
shape_uniformity = tf.feature_column.numeric_column("shape_uniformity");
marginal_adhesion = tf.feature_column.numeric_column("marginal_adhesion");
epithelial_size = tf.feature_column.numeric_column("epithelial_size");
bare_nucleoli = tf.feature_column.numeric_column("bare_nucleoli");
bland_chromatin = tf.feature_column.numeric_column("bland_chromatin");
normal_nucleoli = tf.feature_column.numeric_column("normal_nucleoli");
mitoses = tf.feature_column.numeric_column("mitoses");
feature_columns = [clump_thickness, size_uniformity, shape_uniformity, marginal_adhesion, epithelial_size, 
                   bare_nucleoli, bland_chromatin, normal_nucleoli, mitoses];
feature_spec = tf.feature_column.make_parse_example_spec(feature_columns);
export_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec);
model.export_savedmodel(OUTDIR, export_input_fn, as_text=False)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: ['regression']
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Restoring parameters from ./logs/breastCancer_trained/model.ckpt-12540
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b"./logs/breastCancer_trained/temp-b'1538762567'/saved_model.pb"


b'./logs/breastCancer_trained/1538762567'

In [23]:
# Load model from export directory, and make a predict function.
predict_fn = tf.contrib.predictor.from_saved_model("./logs/breastCancer_trained/1538762567")

# Test inputs represented by Pandas DataFrame.
inputs = pd.DataFrame({
    'clump_thickness': [2,5],
    'size_uniformity': [1,10],
    'shape_uniformity': [1,10],
    'marginal_adhesion': [1,3],
    'epithelial_size': [2,7],
    'bland_chromatin': [1,3],
    'bare_nucleoli': [2,8],
    'normal_nucleoli': [1,10],
    'mitoses': [1,2],
})

# Convert input data into serialized Example strings.
examples = []
for index, row in inputs.iterrows():
    feature = {}
    for col, value in row.iteritems():
        feature[col] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
    example = tf.train.Example(
        features=tf.train.Features(
            feature=feature
        )
    )
    examples.append(example.SerializeToString())

# Make predictions.
predictions = predict_fn({'inputs': examples})

Instructions for updating:
Use the retry module or similar alternatives.
INFO:tensorflow:Restoring parameters from b'./logs/breastCancer_trained/1538762567/variables/variables'


In [24]:
predictions

{'classes': array([[b'0', b'1'],
        [b'0', b'1']], dtype=object),
 'scores': array([[0.9943078 , 0.00569225],
        [0.02123118, 0.9787689 ]], dtype=float32)}